In [ ]:
import sys,os
sys.path.append('../..')
from backtester import matlab, backtester
from backtester.analysis import *
import pandas as pd
import numpy as np
import scipy

%pylab inline
%load_ext autoreload
%autoreload 2

In [ ]:
strategyname = 'strategy_270225'
d, info = matlab.loaddata('../../mat/'+strategyname+'.mat')


In [ ]:
d_slice=d.ix['2000':]

In [ ]:
d_slice.plot()

In [ ]:
info

# Simple entry and exit rules based on MA crossing

In [ ]:
#Defining entry\exit rules
slow_ma = d_slice.exo.rolling(15).mean()#try period == 5
fast_ma = d_slice.exo.rolling(10).mean()#try period == 6


short_entry = CrossDown(fast_ma, slow_ma)
short_exit = (CrossUp(fast_ma, slow_ma)) 

direction = -1
pl, inposition = backtester.backtest(d_slice, short_entry, short_exit, direction )
equity, stats = backtester.stats(pl, inposition)


print(stats)
equity.plot()

# Trailing stop exploration

In [ ]:
figsize(15, 10)


#Trailing stop works better on trending markets.
#On markets where mean-reversion regime prevails, TS will not performing well.
#We need to determine size of TS according to underlying volatility. It will help us to not lose too much on volatility bursts

#Rolling max\min works quite well for raw exploration. Period of rolling window is a subject of optimization
period=5
trailing_stop_short =d_slice.exo.rolling(period).max().shift(1)
trailing_stop_long = d_slice.exo.rolling(period).min().shift(1)

#Simple vola metric. Based on previous px change
# Median smoothing unlike averaging, is less resposive to volatility busts
# I think is will be more useful in our case
vola=d_slice.exo.shift(1)-d_slice.exo.shift(2)
volatility_metric=abs(vola.rolling(10).median())

d_slice.exo.plot()
#slow_ma.plot(c='r')
#fast_ma.plot(c='g')
trailing_stop_short.plot()


#We need to define stop size for median trailing. It must be either fixed value or dynamic value which adjusted by volatility
median_trailstop=d_slice.exo.rolling(5).median().shift(1)
median_trailstop.plot()

In [ ]:
volatility_metric.plot(secondary_y=True,kind='area',stacked=False)

# Optimizing

### Rolling max trailing

In [ ]:
#We're going to optimize period of rolling windows and compare system perfomance  results
#Index of DF is optimizing parameter value

results_a=[]
results_index_a=[]
for p in range(5,200,5):
    optimizing_period=p
    trailing_stop_short =d_slice.exo.rolling(optimizing_period).max().shift(1) 

    short_entry = CrossDown(fast_ma, slow_ma)
    short_exit = (CrossUp(fast_ma, slow_ma)) | (CrossUp(d_slice.exo,trailing_stop_short))

    direction = -1
    pl, inposition = backtester.backtest(d_slice, short_entry, short_exit, direction )
    equity, stats = backtester.stats(pl, inposition)



    results_a.append(stats)
    results_index_a.append(p)


In [ ]:
rollingmax_results_df=pd.DataFrame(results_a,index=results_index_a)

In [ ]:
rollingmax_results_df.sort(['netprofit'],ascending=False)

In [ ]:
optimizing_period=5
trailing_stop_short = pd.rolling_max(d_slice.exo,optimizing_period).shift(1)

short_entry = CrossDown(fast_ma, slow_ma)
short_exit = (CrossUp(fast_ma, slow_ma)) | (CrossUp(d_slice.exo,trailing_stop_short))

direction = -1
pl, inposition = backtester.backtest(d_slice, short_entry, short_exit, direction )
equity, stats = backtester.stats(pl, inposition)

print(stats)
equity.plot()    

#Seems good

In [ ]:
#We're going to optimize period of rolling windows, size of TS and compare systems perfomance  results
#Index of DF is optimizing parameter value

results_a=[]
results_index_a=[]
results_vola_optimization_a=[]

vola=d_slice.exo.shift(1)-d_slice.exo.shift(2)
volatility_metric=abs(vola.rolling(5).median())

for p in range(5,200,5):
    optimizing_period=p
    trailing_stop_short = d_slice.exo.rolling(optimizing_period).max().shift(1)
    
    for v in np.arange(0,2,0.1):
        vola_multiplier=v     
        
        trailing_stop_dynamicsize_short=trailing_stop_short+(volatility_metric*vola_multiplier)
        
        short_entry = CrossDown(fast_ma, slow_ma)
        short_exit = (CrossUp(fast_ma, slow_ma)) | (CrossUp(d_slice.exo,trailing_stop_dynamicsize_short))

        direction = -1
        pl, inposition = backtester.backtest(d_slice, short_entry, short_exit, direction )
        equity, stats = backtester.stats(pl, inposition)

        results_a.append(stats)
        results_index_a.append(p)
        results_vola_optimization_a.append(vola_multiplier)

In [ ]:
rollingmax_dynamicsized_results_df=pd.DataFrame(results_a,index=[results_index_a,results_vola_optimization_a])

In [ ]:
rollingmax_dynamicsized_results_df.sort('netprofit',ascending=False)

In [ ]:
optimizing_period=5

vola_multiplier=0.2

trailing_stop_short = d_slice.exo.rolling(optimizing_period).max().shift(1)

trailing_stop_dynamicsize_short=trailing_stop_short+(volatility_metric*vola_multiplier)

short_entry = CrossDown(fast_ma, slow_ma)
short_exit = (CrossUp(fast_ma, slow_ma)) | (CrossUp(d_slice.exo,trailing_stop_dynamicsize_short))

direction = -1
pl, inposition = backtester.backtest(d_slice, short_entry, short_exit, direction )
equity, stats = backtester.stats(pl, inposition)

print(stats)
equity.plot()    

#performance has improved

## Median trailing stop


### Fixed values

In [ ]:
#We're going to optimize period of rolling windows, size of TS and compare system perfomance  results
#Index of DF is optimizing parameter value

results_a=[]
results_index_a=[]
results_stop_size_a=[]

for p in range(5,200,5):
    optimizing_period=p
    median_trailstop=d_slice.exo.rolling(p).median().shift(1)
    
    for s in range(0,100,5):
        stop_size=s
        
        short_entry = CrossDown(fast_ma, slow_ma)
        short_exit = (CrossUp(fast_ma, slow_ma)) | (CrossUp(d_slice.exo,median_trailstop+stop_size))

        direction = -1
        pl, inposition = backtester.backtest(d_slice, short_entry, short_exit, direction )
        equity, stats = backtester.stats(pl, inposition)

        results_a.append(stats)
        results_index_a.append(p)
        results_stop_size_a.append(stop_size)

In [ ]:
median_results_df=pd.DataFrame(results_a,index=[results_index_a,results_stop_size_a])

In [ ]:
median_results_df.sort(['netprofit'],ascending=False)

In [ ]:
optimizing_period=5
median_trailstop=d_slice.exo.rolling(optimizing_period).median().shift(1)

stop_size=5

short_entry = CrossDown(fast_ma, slow_ma)
short_exit = (CrossUp(fast_ma, slow_ma)) | (CrossUp(d_slice.exo,median_trailstop+stop_size))

direction = -1
pl, inposition = backtester.backtest(d_slice, short_entry, short_exit, direction )
equity, stats = backtester.stats(pl, inposition)

print(stats)
equity.plot()    

#Better than rolling max

## Dynamic vola adjusted value 

In [ ]:
#We're going to optimize period of rolling windows, size of TS and compare system perfomance  results
#Index of DF is optimizing parameter value

results_a=[]
results_index_a=[]
results_stop_size_a=[]
results_vola_multiplier_a=[]

vola=d_slice.exo.shift(1)-d_slice.exo.shift(2)
volatility_metric=abs(vola.rolling(5).median())


for p in range(5,200,5):
    optimizing_period=p
    median_trailstop=d_slice.exo.rolling(p).median().shift(1)
    
    for s in range(0,100,5):
        stop_size=s
        
        for v in np.arange(0,2,0.1):
            vola_multiplier=v 
            short_entry = CrossDown(fast_ma, slow_ma)
            short_exit = (CrossUp(fast_ma, slow_ma)) | (CrossUp(d_slice.exo,median_trailstop+(stop_size*(volatility_metric*v))))

            direction = -1
            pl, inposition = backtester.backtest(d_slice, short_entry, short_exit, direction )
            equity, stats = backtester.stats(pl, inposition)

            results_a.append(stats)
            results_index_a.append(p)
            results_stop_size_a.append(stop_size)
            results_vola_multiplier_a.append(vola_multiplier)

In [ ]:
median_dynamicsize_results_df=pd.DataFrame(results_a,index=[results_index_a,results_stop_size_a,results_vola_multiplier_a])
median_dynamicsize_results_df.sort(['netprofit'],ascending=False)

In [ ]:
optimizing_period=5
median_trailstop=d_slice.exo.rolling(optimizing_period).median().shift(1)

vola=d_slice.exo.shift(1)-d_slice.exo.shift(2)
volatility_metric=abs(vola.rolling(5).median())

vola_multiplier=0.2

stop_size=10

median_trailstop_dynamic=median_trailstop+(stop_size*(volatility_metric*vola_multiplier))

short_entry = CrossDown(fast_ma, slow_ma)
short_exit = (CrossUp(fast_ma, slow_ma)) | (CrossUp(d_slice.exo,median_trailstop_dynamic))

direction = -1
pl, inposition = backtester.backtest(d_slice, short_entry, short_exit, direction )
equity, stats = backtester.stats(pl, inposition)

print(stats)
equity.plot()    

